## Data类——PyG中图的表示及其使用

Data 类对象的创建

- [torch_geometric.data.Data](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data)
- [PyG-01 数据预处理-torch_geometric.data.Data](https://zhuanlan.zhihu.com/p/310008493)

In [1]:
# Data类的构造函数
class Data(object):
    def __init__(self, x=None, edge_index=None, edge_attr=None, y=None, **kwargs):
        pass

从上面代码可以看到 :

x (Tensor, optional): 节点属性矩阵，大小为 [num_nodes, num_node_features]

edge_index (LongTensor, optional): 边索引矩阵，大小为[2, num_edges],第 0 行可称为头（head）节点、源（source）节点、邻接节点，第 1 行可称为尾（tail）节点、目标（target）节点、中心节点

edge_attr (Tensor, optional): 边属性矩阵，大小为 [num_edges, num_edge_features]

y (Tensor, optional): 节点或图的标签，任意大小（，其实也可以是 边的标签）

----

edge_index的每一列定义一条边，其中第一行为边起始节点的索引，第二行为边结 束节点的索引。这种表示方法被称为COO格式（coordinate format），通常用于表示稀疏矩阵。PyG不是用稠密矩阵 来持有邻接矩阵的信息，而是用仅存储邻接矩阵 中非 元素的稀疏矩阵来表示图

通常，一个图至少包含x, edge_index, edge_attr, y, num_nodes5个属性，当图包含其他属性时，我们可以通过指定额外的参数使Data对象包含其他的属性：

graph = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y, num_nodes=num_nodes, other_attr=other_attr)

### 将字典对象转换为图对象

graph_dict中属性值的类型与大小的要求与Data类的构造函数的要求相同。

In [11]:
import numpy as np
from numpy.random import randn
from torch_geometric.data import Data

# 定义节点相关数据
num_nodes = 2000
num_node_features = 4000
num_edges = 100

# 定义字典对象
graph_dict = {
    'x': randn(num_nodes, num_node_features),
    'edge_index': randn(2, num_edges),
    'edge_attr': randn(num_edges, num_node_features),
    'y': randn(num_edges),
    'num_nodes': num_nodes
}

graph_data = Data.from_dict(graph_dict)
print(graph_data)

Data(x=[2000, 4000], edge_index=[2, 100], edge_attr=[100, 4000], y=[100], num_nodes=2000)


### Data对象转换成其他类型数据

我们可以将Data对象转换为dict对象 : def to_dict(self):


或转换为namedtuple: def to_namedtuple(self):

In [12]:
data_dict = Data.to_dict(graph_data)
print(data_dict)

{'x': array([[-0.18383904, -1.27269283, -0.15412519, ...,  1.63401477,
        -0.17532242,  1.94666253],
       [-0.66667775,  0.21520124, -0.22754876, ...,  0.56780496,
        -1.04730827, -0.29289741],
       [ 0.14410923, -0.11216208, -0.52833767, ...,  0.59874052,
         0.18328994, -1.42832704],
       ...,
       [-1.41866019, -0.96120221,  0.05341647, ...,  0.19675187,
         2.17890509,  0.74425714],
       [ 0.13748206,  0.93249476,  2.52298628, ..., -0.36223269,
         1.5105656 , -1.52687794],
       [ 0.38289949, -0.50212704, -0.13747089, ..., -0.53512506,
        -1.72477264, -0.19663631]]), 'edge_index': array([[ 7.51578957e-01,  7.77123854e-01,  5.39323688e-01,
        -8.03574606e-01,  5.63711569e-01,  1.18263288e+00,
         8.51821904e-01, -1.09069428e+00,  8.48613991e-01,
        -5.75848445e-01, -2.00594947e+00,  1.69680727e+00,
        -2.21673881e-01, -2.81111602e-01, -1.25727911e+00,
         2.92224061e-01, -6.47489461e-03,  1.40943514e+00,
         2.2

### Data属性相关

In [13]:
# 获取Data的属性
x = graph_data['x']
print(x.shape)

(2000, 4000)


In [16]:
# 获取Data属性包含的关键字
print(graph_data.keys)

['x', 'edge_index', 'edge_attr', 'y', 'num_nodes']


In [19]:
# 对边排序并移除重复的边
# graph_data.coalesce()

### 图的其他特质

In [21]:
# 导入已有的数据集
from torch_geometric.datasets import KarateClub

datasets = KarateClub()
print("查看图的个数 : {}".format(len(datasets)))

查看图的个数 : 1


In [32]:
# 获取第一张图
data = datasets[0]
print(data)
# 获取图的相关信息
print("图的节点数量 : {}".format(data.num_nodes))
print("图的节点属性的维度 : {}".format(data.num_node_features))
print("同样是节点属性的维度 : {}".format(data.num_features))
print("图的边的数量 : {}".format(data.num_edges))
print("边属性的维度 : {}".format(data.num_edge_features))
print("平均节点度 : {}".format(data.num_edges / data.num_nodes))
print("用作训练集的节点数 {}".format(data.train_mask.sum()))
print("用作训练集的节点数占比 {}".format(int(data.train_mask.sum()) / data.num_nodes))
print("此图是否包含孤立的节点 {}".format(data.contains_isolated_nodes()))
print("此图是否包含自环的边 {}".format(data.contains_self_loops()))
print("此图是否是无向图 {}".format(data.is_undirected()))

Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34])
图的节点数量 : 34
图的节点属性的维度 : 34
同样是节点属性的维度 : 34
图的边的数量 : 156
边属性的维度 : 0
平均节点度 : 4.588235294117647
用作训练集的节点数 4
用作训练集的节点数占比 0.11764705882352941
此图是否包含孤立的节点 False
此图是否包含自环的边 False
此图是否是无向图 True


D:\DevApp\anaconda3\envs\Graph_Network\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  warnings.warn(out)
D:\DevApp\anaconda3\envs\Graph_Network\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'contains_self_loops' is deprecated, use 'has_self_loops' instead
  warnings.warn(out)
